# 0.
1. https://www.bilibili.com/video/BV18g4119737?p=87&spm_id_from=pageDriver&vd_source=70200f7d09862fd682e5f89b22c89125

# 1. rnn layer
## 1.1 构建一个rnn layer
1. 两个参数：
    - `input`: input seq的shape。这里是一个长度为100的word vec,准确的说是“number of input vector”
    - `h_0`: mem 大小，默认初始化为0。这里是一个长度为10的memory,准确的说是“number of hidden state”
    - 第3个参数是“nums of layer”, 默认=1
2. `rnn._parameters.keys()`：查看网络层名称，这里有`l0`的意思是，它也可以像cnn一样有多层，但是不会很长，因为它主要在时间上进行展开


In [1]:
import torch.nn as nn
import torch
rnn = nn.RNN(100,10)
# 查看网络层名称
print(rnn._parameters.keys())

odict_keys(['weight_ih_l0', 'weight_hh_l0', 'bias_ih_l0', 'bias_hh_l0'])


3. 展示weight
    - `weight_ih_l0`连接的是input层和hidden layer,当input也就是$x_t$的shape是`(100,)`时，由$x_t@w_{ih}.T$可得，$w_ih$的shape是`[10, 100]`
    - `weight_hh_l0`链接的是 $h_0$ 和hidden，为了方便计算一般$h_0$的大小会设置成和hidden len的大小。所以$w_hh$的shape是`[10,10]`
4. 展示bias
    - bias直接是hidden len=10

In [3]:
print(rnn.weight_hh_l0.shape,rnn.weight_ih_l0.shape)
print(rnn.bias_hh_l0.shape,rnn.bias_ih_l0.shape)

torch.Size([10, 10]) torch.Size([10, 100])
torch.Size([10]) torch.Size([10])


## 1.2 forward函数
1. `x`：shape=[seq len, b, word vec]，比如是[5,3,100]，3句话，每句话5个单词，每个单词100dim
2. `h0`,`ht`: shape=[num layers, b, h dim]，比如是[1,3, 10]
    - 1是上面定义的lstm单元的层数
    - b是batch size，比如这里可以是3
    - 10是上面hidden units的个数
    - h0是输入，ht是t时刻的输出
3. `out` ：shape=[seq len, b, h dim],比如是[5,3, 10]
4. `ht`和`out`的区别：
    - 前者是最后一个时刻的结果
    - out是运行完了所有时刻/ 所有单词后的“聚合”的结果，是[h1,h2,...,h5]

In [ ]:
out, ht = forward(x, h0)

# 2. Application
## 2.1 Singel layer RNN


In [6]:
rnn = nn.RNN(100,10,1)
print(rnn)


RNN(100, 10)


In [11]:
x = torch.randn(10, 3, 100)
out, h = rnn(x, torch.zeros(1,3,10))
print(out.shape, h.shape)

torch.Size([10, 3, 10]) torch.Size([1, 3, 10])


## 2.2 multi-layer RNN
1. 多层的话，从第2层开始，要考虑上一层的输出
2. 第一层是l0，第二层是l1
    - 第一层把input的embedding-->10 dim
    - 第2层接收第一层的输出，

In [13]:
rnn = nn.RNN(100,10,2)
print(rnn)
print(rnn._parameters.keys())

RNN(100, 10, num_layers=2)
odict_keys(['weight_ih_l0', 'weight_hh_l0', 'bias_ih_l0', 'bias_hh_l0', 'weight_ih_l1', 'weight_hh_l1', 'bias_ih_l1', 'bias_hh_l1'])


In [18]:
rnn.weight_ih_l0.shape,rnn.weight_hh_l0.shape
# [10,100]表明ih是一个100-->10的transition,输入是len=100的vector
# [10,10]表明是一个10-->10的transition，要输出10个单词的表示

(torch.Size([10, 100]), torch.Size([10, 10]))

In [20]:
rnn.weight_ih_l1.shape, rnn.weight_hh_l1.shape

(torch.Size([10, 10]), torch.Size([10, 10]))

- 4层的

In [21]:
rnn = nn.RNN(100,20,4)
print(rnn)

RNN(100, 20, num_layers=4)


- h是最后的时间戳上，所有的状态，所以包括4层的结果：[4,3,20]；
- out中是对10个单词，每个单词取最后一个状态。
- out是最后的时间戳伤，最后的状态，所以是[10, 3, 20],是3句话上10个单词的pred结果

In [24]:
x = torch.randn(10, 3, 100)
out, h = rnn(x)  # h0的shape=[4, 3, 20]
print(out.shape, h.shape)


torch.Size([10, 3, 20]) torch.Size([4, 3, 20])


- 定长的输出：我们的问题是[300, B, 3] ---> [, B, T_Global]
    - T_Global是max(T_i) among all known T_i
- 不定长的输出:我们的问题是[300, B, 3] ---> [, B, T_i]
- 问题1：
    - 这种应用是否常规？比如有一个dim（embedding）其实和时间无关，直接不用？
    - 不常规的用法是否会遭到质疑？
    - 另外输出是一个分布，是否意味着我需要把它归一化？
-

# 3. RNNCEll
1. 其实就是自定义每个单词怎么处理

In [29]:
x = torch.randn(10, 3, 100)

cell1 = nn.RNNCell(100,20)  #
h1 = torch.zeros(3,20)
# 循环10次，从xt-->h1
for xt in x:
    print(xt.shape)
    h1 = cell1(xt,h1)
    print(h1.shape)

torch.Size([3, 100])
torch.Size([3, 20])
torch.Size([3, 100])
torch.Size([3, 20])
torch.Size([3, 100])
torch.Size([3, 20])
torch.Size([3, 100])
torch.Size([3, 20])
torch.Size([3, 100])
torch.Size([3, 20])
torch.Size([3, 100])
torch.Size([3, 20])
torch.Size([3, 100])
torch.Size([3, 20])
torch.Size([3, 100])
torch.Size([3, 20])
torch.Size([3, 100])
torch.Size([3, 20])
torch.Size([3, 100])
torch.Size([3, 20])


## 3.2 two-layer

In [35]:
x = torch.randn(10, 3, 100)

cell1 = nn.RNNCell(100,30)      # layer1: 把100的input-->30的mem
cell2 = nn.RNNCell(30,20)       # layer2: 把30的mem-->20的mem
h1 = torch.zeros(3, 30)
h2 = torch.zeros(3, 20)

for xt in x:
    h1 = cell1(xt,h1)
    h2 = cell2(h1,h2)           # layer2取决于上一个mem的输出，以及自己上一轮的输出

# padding实验

In [76]:
# x = torch.nn.Parameter(torch.reshape(torch.range(1,16),(1,1,4,4)))

x = torch.nn.Parameter(torch.reshape(torch.arange(1,8*8+1,dtype=torch.double),(1,1,8,8)))
conv2 = torch.nn.Conv2d(1,1,(3,3),(4,4),1,bias=False)
x

Parameter containing:
tensor([[[[ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12., 13., 14., 15., 16.],
          [17., 18., 19., 20., 21., 22., 23., 24.],
          [25., 26., 27., 28., 29., 30., 31., 32.],
          [33., 34., 35., 36., 37., 38., 39., 40.],
          [41., 42., 43., 44., 45., 46., 47., 48.],
          [49., 50., 51., 52., 53., 54., 55., 56.],
          [57., 58., 59., 60., 61., 62., 63., 64.]]]], dtype=torch.float64,
       requires_grad=True)

In [77]:
conv2.weight = torch.nn.Parameter(torch.ones(1,1,3,3,dtype=torch.double))

In [78]:
conv2.weight

Parameter containing:
tensor([[[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]]], dtype=torch.float64, requires_grad=True)

In [79]:
ANS = conv2(x)
ANS

tensor([[[[ 22.,  54.],
          [201., 333.]]]], dtype=torch.float64, grad_fn=<ConvolutionBackward0>)

In [67]:
ANS.shape

torch.Size([1, 1, 4, 4])

In [80]:
4+5+6+12+13+14

54